<a href="https://colab.research.google.com/github/SatishKumarAnbalagan/EECE7398_DeepLearningInEmbeddedSystems/blob/master/semantic_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# deep learning project - semantic segmentation

In [0]:
import keras
import numpy as np
from keras import backend as K
from keras.models import Model
import cv2
import glob
import numpy as np
from ASPP import reduce_mean
from bilinear import resize_bilinear
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import load_model

from keras.layers import Conv2D, Dense, Flatten, Input, Activation, MaxPooling2D, BatchNormalization, UpSampling2D, Concatenate, Lambda, SeparableConv2D
from keras.layers.merge import add
from keras.regularizers import l2
from keras.optimizers import Adam
import tensorflow as tf
from tqdm import tqdm

ROW_AXIS = 1
COL_AXIS = 2
CHANNEL_AXIS = 3


def _shortcut(inp, residual):
    input_shape = K.int_shape(inp)
    residual_shape = K.int_shape(residual)
    stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
    stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]

    shortcut = inp
    # 1 X 1 conv if shape is different. Else identity.
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS],
                          kernel_size=(1, 1),
                          strides=(stride_width, stride_height),
                          padding="valid",
                          kernel_initializer="he_normal",
                          kernel_regularizer=l2(0.0001))(inp)
    # print(shortcut.shape)
    z = add([shortcut, residual])
    return z


def ASPP(network):
    image_features = network
    # feature_map_size = [32, 40]
    # print(feature_map_size)
    # a = 1
    # image_features = reduce_mean(a)(network)
    # print(image_features.shape)
    # image_features = Conv2D(256, (1, 1))(image_features)
    image_features = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001)(image_features)
    # image_features = resize_bilinear(feature_m
    # ap_size[0], feature_map_size[1])(image_features)
    # print(image_features.shape)

    at_pool1x1 = Conv2D(256, (1, 1), padding='same')(network)

    at_pool3x3_1 = Conv2D(256, (3, 3), dilation_rate=6, padding='same')(network)

    at_pool3x3_2 = Conv2D(256, (3, 3), dilation_rate=12, padding='same')(network)

    at_pool3x3_3 = Conv2D(256, (3, 3), dilation_rate=18, padding='same')(network)
    net = [image_features, at_pool1x1, at_pool3x3_1, at_pool3x3_2, at_pool3x3_3]

    def cnct(net):
        return K.concatenate([net[0], net[1], net[2], net[3], net[4]], axis=-1)

    network = Lambda(cnct)(net)
    network = SeparableConv2D(256, (1, 1), padding='same')(network)
    network = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001)(network)

    return network


# def resnet(z):
model_inp = Input(shape=(750, 1100, 3))
# BLOCK_1

x = Conv2D(16, (7, 7), strides=(2, 2), padding='same', name='block1_conv1')(model_inp)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block1_BN1')(x)
x = Activation('relu')(x)
y_down1 = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same', name='block1_pool1')(x)

# BLOCK_2

# branch1
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block2_branch1_BN1')(y_down1)
x = Activation('relu')(x)
x_down2 = Conv2D(16, (1, 1), strides=(2, 2), padding='same', name='block2_branch1_conv1')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block2_branch1_BN2')(x_down2)
x = Activation('relu')(x)
x = Conv2D(16, (3, 3), strides=(1, 1), padding='same', name='block2_branch1_conv2')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block2_branch1_BN3')(x)
x = Activation('relu')(x)
x = Conv2D(64, (1, 1), strides=(1, 1), padding='same', name='block2_branch1_conv3')(x)
y = _shortcut(y_down1, x)

# branch2
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block2_branch2_BN1')(y)
x = Activation('relu')(x)
x = Conv2D(16, (1, 1), strides=(1, 1), padding='same', name='block2_branch2_conv1')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block2_branch2_BN2')(x)
x = Activation('relu')(x)
x = Conv2D(16, (3, 3), strides=(1, 1), padding='same', name='block2_branch2_conv2')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block2_branch2_BN3')(x)
x = Activation('relu')(x)
x = Conv2D(64, (1, 1), strides=(1, 1), padding='same', name='block2_branch2_conv3')(x)
y = _shortcut(y, x)

# branch3
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block2_branch3_BN1')(y)
x = Activation('relu')(x)
x = Conv2D(16, (1, 1), strides=(1, 1), padding='same', name='block2_branch3_conv1')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block2_branch3_BN2')(x)
x = Activation('relu')(x)
x = Conv2D(16, (3, 3), strides=(1, 1), padding='same', name='block2_branch3_conv2')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block2_branch3_BN3')(x)
x = Activation('relu')(x)
x = Conv2D(64, (1, 1), strides=(1, 1), padding='same', name='block2_branch3_conv3')(x)
y = _shortcut(y, x)

# BLOCK_3

# branch1
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block3_branch1_BN1')(y)
x = Activation('relu')(x)
x_down3 = Conv2D(32, (1, 1), strides=(2, 2), padding='same', name='block3_branch1_conv1')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block3_branch1_BN2')(x_down3)
x = Activation('relu')(x)
x = Conv2D(32, (3, 3), strides=(1, 1), padding='same', name='block3_branch1_conv2')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block3_branch1_BN3')(x)
x = Activation('relu')(x)
x = Conv2D(128, (1, 1), strides=(1, 1), padding='same', name='block3_branch1_conv3')(x)
y = _shortcut(y, x)

# branch2
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block3_branch2_BN1')(y)
x = Activation('relu')(x)
x = Conv2D(32, (1, 1), strides=(1, 1), padding='same', name='block3_branch2_conv1')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block3_branch2_BN2')(x)
x = Activation('relu')(x)
x = Conv2D(32, (3, 3), strides=(1, 1), padding='same', name='block3_branch2_conv2')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block3_branch2_BN3')(x)
x = Activation('relu')(x)
x = Conv2D(128, (1, 1), strides=(1, 1), padding='same', name='block3_branch2_conv3')(x)
y = _shortcut(y, x)
# branch3
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block3_branch3_BN1')(y)
x = Activation('relu')(x)
x = Conv2D(32, (1, 1), strides=(1, 1), padding='same', name='block3_branch3_conv1')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block3_branch3_BN2')(x)
x = Activation('relu')(x)
x = Conv2D(32, (3, 3), strides=(1, 1), padding='same', name='block3_branch3_conv2')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block3_branch3_BN3')(x)
x = Activation('relu')(x)
x = Conv2D(128, (1, 1), strides=(1, 1), padding='same', name='block3_branch3_conv3')(x)
y = _shortcut(y, x)
# branch4
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block3_branch4_BN1')(y)
x = Activation('relu')(x)
x = Conv2D(32, (1, 1), strides=(1, 1), padding='same', name='block3_branch4_conv1')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block3_branch4_BN2')(x)
x = Activation('relu')(x)
x = Conv2D(32, (3, 3), strides=(1, 1), padding='same', name='block3_branch4_conv2')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block3_branch4_BN3')(x)
x = Activation('relu')(x)
x = Conv2D(128, (1, 1), strides=(1, 1), padding='same', name='block3_branch4_conv3')(x)
y = _shortcut(y, x)

# BLOCK_4

# branch1
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block4_branch1_BN1')(y)
x = Activation('relu')(x)
x = Conv2D(64, (1, 1), strides=(1, 1), padding='same', name='block4_branch1_conv1')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block4_branch1_BN2')(x)
x = Activation('relu')(x)
x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', name='block4_branch1_conv2')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block4_branch1_BN3')(x)
x = Activation('relu')(x)
x = Conv2D(256, (1, 1), strides=(1, 1), padding='same', name='block4_branch1_conv3')(x)
y = _shortcut(y, x)
# branch2
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block4_branch2_BN1')(y)
x = Activation('relu')(x)
x = Conv2D(64, (1, 1), strides=(1, 1), padding='same', name='block4_branch2_conv1')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block4_branch2_BN2')(x)
x = Activation('relu')(x)
x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', name='block4_branch2_conv2')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block4_branch2_BN3')(x)
x = Activation('relu')(x)
x = Conv2D(256, (1, 1), strides=(1, 1), padding='same', name='block4_branch2_conv3')(x)
y = _shortcut(y, x)

# branch3
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block4_branch3_BN1')(y)
x = Activation('relu')(x)
x = Conv2D(64, (1, 1), strides=(1, 1), padding='same', name='block4_branch3_conv1')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block4_branch3_BN2')(x)
x = Activation('relu')(x)
x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', name='block4_branch3_conv2')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block4_branch3_BN3')(x)
x = Activation('relu')(x)
x = Conv2D(256, (1, 1), strides=(1, 1), padding='same', name='block4_branch3_conv3')(x)
y = _shortcut(y, x)
# branch4
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block4_branch4_BN1')(y)
x = Activation('relu')(x)
x = Conv2D(64, (1, 1), strides=(1, 1), padding='same', name='block4_branch4_conv1')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block4_branch4_BN2')(x)
x = Activation('relu')(x)
x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', name='block4_branch4_conv2')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block4_branch4_BN3')(x)
x = Activation('relu')(x)
x = Conv2D(256, (1, 1), strides=(1, 1), padding='same', name='block4_branch4_conv3')(x)
y = _shortcut(y, x)
# branch5
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block4_branch5_BN1')(y)
x = Activation('relu')(x)
x = Conv2D(64, (1, 1), strides=(1, 1), padding='same', name='block4_branch5_conv1')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block4_branch5_BN2')(x)
x = Activation('relu')(x)
x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', name='block4_branch5_conv2')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block4_branch5_BN3')(x)
x = Activation('relu')(x)
x = Conv2D(256, (1, 1), strides=(1, 1), padding='same', name='block4_branch5_conv3')(x)
y = _shortcut(y, x)
# branch6
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block4_branch6_BN1')(y)
x = Activation('relu')(x)
x = Conv2D(64, (1, 1), strides=(1, 1), padding='same', name='block4_branch6_conv1')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block4_branch6_BN2')(x)
x = Activation('relu')(x)
x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', name='block4_branch6_conv2')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block4_branch6_BN3')(x)
x = Activation('relu')(x)
x = Conv2D(256, (1, 1), strides=(1, 1), padding='same', name='block4_branch6_conv3')(x)
y = _shortcut(y, x)

# BLOCK_5_ATROUS

# branch1
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block5_branch1_BN1')(y)
x = Activation('relu')(x)
x = Conv2D(128, (1, 1), strides=(1, 1), padding='same', dilation_rate=2, name='block5_branch1_conv1')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block5_branch1_BN2')(x)
x = Activation('relu')(x)
x = Conv2D(128, (3, 3), strides=(1, 1), padding='same', dilation_rate=4, name='block5_branch1_conv2')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block5_branch1_BN3')(x)
x = Activation('relu')(x)
x = Conv2D(512, (1, 1), strides=(1, 1), padding='same', dilation_rate=2, name='block5_branch1_conv3')(x)
y = _shortcut(y, x)

# branch2
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block5_branch2_BN1')(y)
x = Activation('relu')(x)
x = Conv2D(128, (1, 1), strides=(1, 1), padding='same', dilation_rate=2, name='block5_branch2_conv1')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block5_branch2_BN2')(x)
x = Activation('relu')(x)
x = Conv2D(128, (3, 3), strides=(1, 1), padding='same', dilation_rate=4, name='block5_branch2_conv2')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block5_branch2_BN3')(x)
x = Activation('relu')(x)
x = Conv2D(512, (1, 1), strides=(1, 1), padding='same', dilation_rate=2, name='block5_branch2_conv3')(x)
y = _shortcut(y, x)

# branch3
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block5_branch3_BN1')(y)
x = Activation('relu')(x)
x = Conv2D(128, (1, 1), strides=(1, 1), padding='same', dilation_rate=2, name='block5_branch3_conv1')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block5_branch3_BN2')(x)
x = Activation('relu')(x)
x = Conv2D(128, (3, 3), strides=(1, 1), padding='same', dilation_rate=4, name='block5_branch3_conv2')(x)
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001, name='block5_branch3_BN3')(x)
x = Activation('relu')(x)
x = Conv2D(512, (1, 1), strides=(1, 1), padding='same', dilation_rate=2, name='block5_branch3_conv3')(x)
y = _shortcut(y, x)


Resnet = y
network = ASPP(Resnet)

network = UpSampling2D(size=(2, 2), name='US1')(network)
network = Conv2D(64, (3, 3), activation='relu', strides=(1, 1), name='up1')(network)

Low_f8x = SeparableConv2D(48, (1, 1), padding='same')(x_down2)


def concat(listt):
    return K.concatenate(listt, axis=-1)


combine_1 = [Low_f8x, network]

network = Lambda(concat)(combine_1)

network = UpSampling2D(size=(2, 2), name='US1')(network)
network = Conv2D(64, (3, 3), activation='relu', strides=(1, 1), name='up1')(network)

Low_f4x = SeparableConv2D(48, (1, 1), padding='same')(y_down1)

combine_2 = [Low_f4x, network]

network = Lambda(concat)(combine_2)

network = UpSampling2D(size=(2, 2), name='US1')(network)
network = Conv2D(32, (3, 3), activation='relu', strides=(1, 1), name='up1')(network)

network = UpSampling2D(size=(2, 2), name='US1')(network)
network = Conv2D(16, (3, 3), activation='relu', strides=(1, 1), name='up1')(network)

'''

network = Conv2D(64, (1, 1), strides=(1, 1), activation='relu', name='Final_Conv_1')(network)
network = Conv2D(32, (1, 1), strides=(1, 1), activation='relu', name='Final_Conv_2')(network)
print(network.shape)
network_up1 = UpSampling2D(size=(2, 2), name='Upsample_1')(network)
network_up1 = Conv2D(16, (3, 3), activation='relu', strides=(1, 1), name='up1')(network_up1)
reshape_1 = resize_bilinear(750, 1100)(network_up1)
network_up2 = UpSampling2D(size=(2, 2), name='Upsample_2')(network_up1)
network_up2 = Conv2D(16, (3, 3), activation='relu', strides=(1, 1), name='up2')(network_up2)
reshape_2 = resize_bilinear(750, 1100)(network_up2)
network_up3 = UpSampling2D(size=(2, 2), name='Upsample_3')(network_up2)
network_up3 = Conv2D(16, (3, 3), activation='relu', strides=(1, 1), name='up3')(network_up3)
reshape_3 = resize_bilinear(750, 1100)(network_up3)
network_up4 = UpSampling2D(size=(2, 2), name='Upsample_4')(network_up3)
network_up4 = Conv2D(16, (3, 3), activation='relu', strides=(1, 1), name='up4')(network_up4)
reshape_4 = resize_bilinear(750, 1100)(network_up4)

res = [reshape_1, reshape_2, reshape_3, reshape_4]


def cnkt(res):
    return K.concatenate([res[0], res[1], res[2], res[3]], axis=-1)


#network = add([reshape_1, reshape_2, reshape_3, reshape_4])
network = Lambda(cnkt)(res)
'''  # print(network.shape)
network = Conv2D(8, (3, 3), strides=(1, 1), padding='same', name='before_output')(network)
network = Conv2D(2, (3, 3), activation='softmax', strides=(1, 1), padding='same', name='Final_Output')(network)
# print(network.shape)
Optimizer = Adam(lr=0.00001)

resnet_encoder = Model(inputs=model_inp, outputs=network)
resnet_encoder.compile(optimizer=Optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint_val = ModelCheckpoint('Segmentation_val_loss.{val_loss:.4f}-{epoch:03d}.h5', monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=2)
checkpoint_loss = ModelCheckpoint('Segmentation_loss.{loss:.4f}-{epoch:03d}.h5', monitor='loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=3)

photo = TensorBoard(log_dir='logs')
#resnet_encoder = load_model('Segmentation_val_loss.0.1845-028.h5', custom_objects={'reduce_mean': reduce_mean, 'resize_bilinear': resize_bilinear})

resnet_encoder.fit(x=train_img, y=train_mask, epochs=700, batch_size=4, verbose=1, callbacks=[checkpoint_loss, checkpoint_val, photo], validation_data=(test_img, test_mask), shuffle=True)
